In [3]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import  KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.feature_selection import RFE
import operator
import seaborn as sn
from scipy import stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_predict
from imblearn.over_sampling import SVMSMOTE
from sklearn.model_selection import cross_validate
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import BernoulliNB
import pandasql as ps
from sklearn.model_selection import RepeatedStratifiedKFold
import pandasql as ps
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [6]:
df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/TRAIN_CLEARED2.csv")
df = df.drop(['Unnamed: 0'], axis=1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11028 entries, 0 to 11027
Data columns (total 48 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   NewName                                      11028 non-null  object 
 1   ClubInvolved                                 11028 non-null  object 
 2   League                                       11028 non-null  object 
 3   MainPosition                                 11028 non-null  object 
 4   LeagueInvolved                               11028 non-null  object 
 5   CorrMarketValue                              11028 non-null  float64
 6   Country                                      11028 non-null  object 
 7   Club                                         11028 non-null  object 
 8   Position                                     11028 non-null  object 
 9   Foot                                         11028 non-null  object 
 10

In [198]:
df = df.drop(['NewName', 'ClubInvolved', 'Club', 'ClubAge', 'ClubNumPlayersUsedPerMatch',\
     'ClubGoalPerShotOnTargetPerMatch', 'ClubShotOnTargetAgainstUsedPerMatch', 'ClubPointsPerMatch', 'Fee'], axis = 1)

In [4]:
df.columns

Index(['League', 'MainPosition', 'LeagueInvolved', 'CorrMarketValue',
       'Country', 'Position', 'Foot', 'IntReputation', 'Heading_accuracy',
       'Reactions', 'Balance', 'Jumping', 'Season', 'Years', 'DEGREE', 'Eigv',
       'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA',
       'ClubInvolvedAge', 'ClubInvolvedCleanSheetsRate', 'MinutesPlayedPer90',
       'ContractLeft', 'Pace_main', 'Shooting_main', 'Defending_main',
       'Passing_main', 'Physical_main', 'Age', 'ClubInvolvedPointsPerMatch',
       'ClubInvolvedNumPlayersUsedPerMatch',
       'ClubInvolvedGoalPerShotOnTargetPerMatch',
       'ClubInvolvedShotOnTargetAgainstUsedPerMatch', 'GoalsPerMinutesPlayed',
       'AssistsPerMinutesPlayed', 'YellowCardsPerMinutesPlayed',
       'RedCardsPerMinutesPlayed'],
      dtype='object')

In [199]:
df['target'] = 0
df.loc[df.League == 'Spain Primera Division', 'target'] = 1

In [200]:
df = df.drop('League', axis=1)

In [201]:
df['target'].value_counts()

0    10034
1      994
Name: target, dtype: int64

In [202]:
#set feature list and target
feature_list = list(df.columns.values)
feature_list.remove('target')
target = 'target'

# define dataset
X = df[feature_list]
y = df[target]

In [203]:
import category_encoders as ce
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

In [204]:
#Impute missing values
imputer = IterativeImputer()
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [155]:
#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

In [205]:
X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']
# X_imp_st['Foot_Reactions'] = X_imp_st['Foot'] + X_imp_st['Reactions']

In [206]:

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

[16:39:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:39:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:39:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:39:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[16:39:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:39:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.war

...

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this w

[16:41:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:41:09] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:41:09] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:41:09] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 20.42418929338455
score_time 0.10276237726211548
test_acc 0.9286805659536028
test_recall 0.36709595959595964
test_auc 0.9021223551366905
test_precison 0.7002358523785333
test_f1 0.48005381735092234


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [207]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 20.42418929338455
score_time 0.10276237726211548
test_acc 0.9286805659536028
test_recall 0.36709595959595964
test_auc 0.9021223551366905
test_precison 0.7002358523785333
test_f1 0.48005381735092234


fit_time 10.696826362609864
score_time 0.05490661859512329
test_acc 0.9278646506064142
test_recall 0.3666565656565657
test_auc 0.8997688448598428
test_precison 0.6876294072540907
test_f1 0.47733104949611516

Different imputers

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RepeatedStratifiedKFold

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator = DecisionTreeRegressor(max_features="sqrt", random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

[10:24:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:24:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:24:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:24:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[10:24:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:24:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:24:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:24:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarnin

...

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarnin

[10:25:21] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 11.878182721138
score_time 0.07854596376419068
test_acc 0.9281825840431885
test_recall 0.3666565656565657
test_auc 0.8984826468720524
test_precison 0.6930480615940607
test_f1 0.47826222493757753


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [29]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  KNeighborsRegressor(n_neighbors=15))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

[08:14:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:14:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:14:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:14:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[08:14:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:14:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:14:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarnin

[08:15:30] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 10.172429537773132
score_time 0.07240152359008789
test_acc 0.9260960044623392
test_recall 0.35048484848484857
test_auc 0.899291929856533
test_precison 0.6732127233482041
test_f1 0.4599679939934587


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

In [9]:

from sklearn.ensemble import GradientBoostingRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  GradientBoostingRegressor(random_state = 1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

KeyboardInterrupt: 

In [12]:
from sklearn.ensemble import HistGradientBoostingRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  HistGradientBoostingRegressor(random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

[10:38:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:38:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:38:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:38:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[10:39:07] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:39:07] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:39:07] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:39:07] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this w

...

[10:39:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:39:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:39:40] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:39:40] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 11.88404415845871
score_time 0.06365435123443604
test_acc 0.9271844811954859
test_recall 0.3635656565656566
test_auc 0.8964119165301792
test_precison 0.6790851416363608
test_f1 0.47218737566373276


In [13]:

from sklearn.ensemble import GradientBoostingRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  LGBMRegressor(random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

[10:45:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:45:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:45:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:45:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[10:45:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:45:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:45:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:45:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

...

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarnin

[10:45:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:45:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:45:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


fit_time 10.157270765304565
score_time 0.05799556970596313
test_acc 0.9275922948961174
test_recall 0.35953535353535354
test_auc 0.8971199541541981
test_precison 0.6867498146833838
test_f1 0.47036362516540897


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

In [14]:

from sklearn.ensemble import GradientBoostingRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  XGBRegressor(random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

[11:13:30] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:13:30] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:13:30] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:13:30] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[11:13:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:13:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:13:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:13:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

...

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:14:05] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:14:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


fit_time 8.753902697563172
score_time 0.05635489225387573
test_acc 0.9268211345727624
test_recall 0.35656565656565653
test_auc 0.8999469122713553
test_precison 0.6831893576069913
test_f1 0.46620834046238196


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

In [15]:

from xgboost import XGBRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  XGBRegressor(random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

[11:38:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:38:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:38:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:38:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[11:38:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:38:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:38:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[11:38:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:38:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:38:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 9.484078204631805
score_time 0.059662795066833495
test_acc 0.9268211345727624
test_recall 0.35656565656565653
test_auc 0.8999469122713553
test_precison 0.6831893576069913
test_f1 0.46620834046238196


DecisionTreeRegressor hyperparameter tuning

In [208]:
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator = DecisionTreeRegressor(max_depth= None, max_features="sqrt", random_state=1, \
    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.2, min_impurity_decrease=0.))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']


In [15]:
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator = DecisionTreeRegressor(max_depth= None, max_features="sqrt", random_state=1, \
    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.2, min_impurity_decrease=0.))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 10.200668835639954
score_time 0.07004737854003906
test_acc 0.9286805659536028
test_recall 0.36709595959595964
test_auc 0.9021223551366905
test_precison 0.7002358523785333
test_f1 0.48005381735092234


In [16]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 10.200668835639954
score_time 0.07004737854003906
test_acc 0.9286805659536028
test_recall 0.36709595959595964
test_auc 0.9021223551366905
test_precison 0.7002358523785333
test_f1 0.48005381735092234


XGBoost hyperparameter tuning

In [209]:
# define the model with default hyperparameters
model = XGBClassifier()
# define the grid of values to search
grid = dict()
grid['verbosity'] = [0]
grid['random_state'] = [1]
grid['n_estimators'] = [500] #100, 300
grid['learning_rate'] = [0.3] #0.01, 0.1, 0.2 , 0.3
grid['subsample'] = [1] #0.5, 0.75
# grid['max_features'] = [10, 20, 30]
grid['max_depth'] = [6] #3,9
#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit='f1')
# execute the grid search
grid_result = grid_search.fit(X_imp_st, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

Best: 0.509750 using {'learning_rate': 0.3, 'max_depth': 6, 'n_estimators': 500, 'random_state': 1, 'subsample': 1, 'verbosity': 0}


In [211]:
#create df for metrics
columns= ['params', 'mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']
indexes = [x for x in range(len(grid_result.cv_results_['params']))]
metric_df = pd.DataFrame(index = indexes, columns=columns)
metric_df = metric_df.fillna(0)
metric_df['params'] = metric_df['params'].astype(str)
metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']] = metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']].astype(float)

metric_df['params'] = grid_result.cv_results_['params']
metric_df['mean_test_acc'] = grid_result.cv_results_['mean_test_acc']
metric_df['mean_test_recall'] = grid_result.cv_results_['mean_test_recall']
metric_df['mean_test_auc'] = grid_result.cv_results_['mean_test_auc']
metric_df['mean_test_precison'] = grid_result.cv_results_['mean_test_precison']
metric_df['mean_test_f1'] = grid_result.cv_results_['mean_test_f1']

metric_df.sort_values(by='mean_test_f1', ascending=False)

,params,mean_test_acc,mean_test_auc,mean_test_recall,mean_test_precison,mean_test_f1
0,"{'learning_rate': 0.3, 'max_depth': 6, 'n_esti...",0.930993,0.904806,0.400298,0.705767,0.50975


In [68]:
pd.set_option('display.max_colwidth', None)

In [17]:
# define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 77.93723344802856
score_time 0.09060708284378052
test_acc 0.9299049120284064
test_recall 0.39480808080808083
test_auc 0.9036096952211496
test_precison 0.6984657035009281
test_f1 0.50272548108979


In [18]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 77.93723344802856
score_time 0.09060708284378052
test_acc 0.9299049120284064
test_recall 0.39480808080808083
test_auc 0.9036096952211496
test_precison 0.6984657035009281
test_f1 0.50272548108979


In [19]:
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)
m = model.fit(X_imp_st , y)
zip_iterator = zip(list(X_imp_st.columns), m.feature_importances_)
feature_dict = dict(zip_iterator)
print(dict( sorted(feature_dict.items(), key=operator.itemgetter(1),reverse=True)))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


{'SP_LA_LIGA': 0.21124952, 'Country': 0.056802247, 'FR_LIGUE_1': 0.036309894, 'IT_SERIE_A': 0.03377632, 'LeagueInvolved': 0.027453369, 'Foot': 0.025986897, 'CorrMarketValue': 0.02563341, 'GE_BUNDESLIGA': 0.025446309, 'ContractLeft': 0.022842059, 'LeagueInvolved_CorrMarketValue': 0.020756476, 'Passing_main': 0.018951163, 'ClubInvolvedPointsPerMatch': 0.018767461, 'DEGREE': 0.018724125, 'ClubInvolvedAge': 0.018708609, 'Jumping': 0.018436039, 'IntReputation': 0.018365737, 'EN_PL': 0.018146373, 'ClubInvolvedGoalPerShotOnTargetPerMatch': 0.01777319, 'ClubInvolvedCleanSheetsRate': 0.017522674, 'Reactions': 0.017324643, 'ClubInvolvedNumPlayersUsedPerMatch': 0.01728091, 'Age': 0.01692151, 'Defending_main': 0.01678371, 'Season_SP_LA_LIGA': 0.016673401, 'MinutesPlayedPer90': 0.016526239, 'MainPosition': 0.016420266, 'ClubInvolvedShotOnTargetAgainstUsedPerMatch': 0.016148973, 'LeagueInvolved_Season': 0.015696976, 'Position': 0.015547953, 'Years': 0.015380169, 'LeagueInvolved_MinutesPlayedPer90': 

Stacking

In [78]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import BernoulliNB

In [19]:
# define the model
model = LogisticRegression()

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 0.5363524556159973
score_time 0.05408011674880982
test_acc 0.9184801226814182
test_recall 0.2998030303030303
test_auc 0.8797734874775248
test_precison 0.600474561682533
test_f1 0.39798016809433967


In [20]:
# define the model
model = KNeighborsClassifier()

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 0.03650875091552734
score_time 1.8868317604064941
test_acc 0.91680209723358
test_recall 0.29364646464646466
test_auc 0.8020361860613179
test_precison 0.5768771601529821
test_f1 0.38734475550850733


In [114]:
# define the model with default hyperparameters
model =  KNeighborsClassifier()
# define the grid of values to search
grid = dict()

grid['n_neighbors'] = [5] #, 7, 9, 11, 13
grid['weights'] = ['uniform', 'distance']

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit='f1')
# execute the grid search
grid_result = grid_search.fit(X_imp_st, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.390328 using {'n_neighbors': 5, 'weights': 'distance'}


In [115]:
#create df for metrics
columns= ['params', 'mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']
indexes = [x for x in range(len(grid_result.cv_results_['params']))]
metric_df = pd.DataFrame(index = indexes, columns=columns)
metric_df = metric_df.fillna(0)
metric_df['params'] = metric_df['params'].astype(str)
metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']] = metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']].astype(float)

metric_df['params'] = grid_result.cv_results_['params']
metric_df['mean_test_acc'] = grid_result.cv_results_['mean_test_acc']
metric_df['mean_test_recall'] = grid_result.cv_results_['mean_test_recall']
metric_df['mean_test_auc'] = grid_result.cv_results_['mean_test_auc']
metric_df['mean_test_precison'] = grid_result.cv_results_['mean_test_precison']
metric_df['mean_test_f1'] = grid_result.cv_results_['mean_test_f1']

metric_df.sort_values(by='mean_test_f1', ascending=False)

,params,mean_test_acc,mean_test_auc,mean_test_recall,mean_test_precison,mean_test_f1
1,"{'n_neighbors': 5, 'weights': 'distance'}",0.917029,0.804470,0.296662,0.578440,0.390328
0,"{'n_neighbors': 5, 'weights': 'uniform'}",0.916802,0.802036,0.293646,0.576877,0.387345


In [21]:
# define the model
model = DecisionTreeClassifier()

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 1.230445396900177
score_time 0.039172124862670896
test_acc 0.8870599157881573
test_recall 0.4013181818181818
test_auc 0.6682442835383744
test_precison 0.3803180951786852
test_f1 0.3900889461808464


In [107]:
# define the model with default hyperparameters
model =  DecisionTreeClassifier()
# define the grid of values to search
grid = dict()

grid['random_state'] = [1]
grid['criterion'] = ['entropy'] #'gini'
# grid['learning_rate'] = [0.5]#0.1, 1
# # grid['max_samples'] = [0.7, 1] #0.5, 0.75
grid['max_features'] = [ None] #, 'auto', 'sqrt', 'log2', 10, 20, 30
grid['max_depth'] = [ 21, ] #11, 13, 15, 17, 19, None, 23, 25, 27
#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit='f1')
# execute the grid search
grid_result = grid_search.fit(X_imp_st, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.416411 using {'criterion': 'entropy', 'max_depth': 21, 'max_features': None, 'random_state': 1}


In [109]:
#create df for metrics
columns= ['params', 'mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']
indexes = [x for x in range(len(grid_result.cv_results_['params']))]
metric_df = pd.DataFrame(index = indexes, columns=columns)
metric_df = metric_df.fillna(0)
metric_df['params'] = metric_df['params'].astype(str)
metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']] = metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']].astype(float)

metric_df['params'] = grid_result.cv_results_['params']
metric_df['mean_test_acc'] = grid_result.cv_results_['mean_test_acc']
metric_df['mean_test_recall'] = grid_result.cv_results_['mean_test_recall']
metric_df['mean_test_auc'] = grid_result.cv_results_['mean_test_auc']
metric_df['mean_test_precison'] = grid_result.cv_results_['mean_test_precison']
metric_df['mean_test_f1'] = grid_result.cv_results_['mean_test_f1']

metric_df.sort_values(by='mean_test_f1', ascending=False)

,params,mean_test_acc,mean_test_auc,mean_test_recall,mean_test_precison,mean_test_f1
0,"{'criterion': 'entropy', 'max_depth': 21, 'max_features': None, 'random_state': 1}",0.893453,0.684249,0.422970,0.411675,0.416411
6,"{'criterion': 'entropy', 'max_depth': 21, 'max_features': 30, 'random_state': 1}",0.891367,0.682950,0.420884,0.402586,0.410248
4,"{'criterion': 'entropy', 'max_depth': 21, 'max_features': 10, 'random_state': 1}",0.889780,0.665677,0.379207,0.386552,0.382437
1,"{'criterion': 'entropy', 'max_depth': 21, 'max_features': 'auto', 'random_state': 1}",0.890098,0.663877,0.375202,0.389318,0.381094
2,"{'criterion': 'entropy', 'max_depth': 21, 'max_features': 'sqrt', 'random_state': 1}",0.890098,0.663877,0.375202,0.389318,0.381094
5,"{'criterion': 'entropy', 'max_depth': 21, 'max_features': 20, 'random_state': 1}",0.886107,0.662129,0.380202,0.372192,0.375326
3,"{'criterion': 'entropy', 'max_depth': 21, 'max_features': 'log2', 'random_state': 1}",0.886651,0.655760,0.363015,0.369900,0.364839


In [22]:
# define the model
model = SVC()

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 8.092010247707368
score_time 4.659827518463135
test_acc 0.9243284278317013
test_recall 0.2861666666666667
test_auc 0.8571782714755647
test_precison 0.7001570921754722
test_f1 0.4045057829128055


In [97]:
# define the model with default hyperparameters
model = SVC()
# define the grid of values to search
grid = dict()

grid['C'] = [10] #0.1, 1, 5, 10,  20, 40, 60
grid['kernel'] = ['rbf']#, 'poly', 'rbf', 'sigmoid', 'precomputed'] #100, 300
grid['gamma'] = ['scale']#, 'auto'
# # grid['max_samples'] = [0.7, 1] #0.5, 0.75
# grid['max_features'] = [ None] #10, sqrt, 20, 40
# grid['max_depth'] = [13] #3,6, 9, 11, 17, 21
#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit='f1')
# execute the grid search
grid_result = grid_search.fit(X_imp_st, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.473265 using {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}


In [98]:
#create df for metrics
columns= ['params', 'mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']
indexes = [x for x in range(len(grid_result.cv_results_['params']))]
metric_df = pd.DataFrame(index = indexes, columns=columns)
metric_df = metric_df.fillna(0)
metric_df['params'] = metric_df['params'].astype(str)
metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']] = metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']].astype(float)

metric_df['params'] = grid_result.cv_results_['params']
metric_df['mean_test_acc'] = grid_result.cv_results_['mean_test_acc']
metric_df['mean_test_recall'] = grid_result.cv_results_['mean_test_recall']
metric_df['mean_test_auc'] = grid_result.cv_results_['mean_test_auc']
metric_df['mean_test_precison'] = grid_result.cv_results_['mean_test_precison']
metric_df['mean_test_f1'] = grid_result.cv_results_['mean_test_f1']

metric_df.sort_values(by='mean_test_f1', ascending=False)

,params,mean_test_acc,mean_test_auc,mean_test_recall,mean_test_precison,mean_test_f1
0,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.922878,0.859160,0.387687,0.614898,0.473265
1,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.917936,0.856114,0.372167,0.570376,0.448218


In [23]:
# define the model
model = GaussianNB()

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 0.05277284383773804
score_time 0.05560401678085327
test_acc 0.8813012440909382
test_recall 0.5527676767676768
test_auc 0.83910367897274
test_precison 0.38966719201022215
test_f1 0.45670530680654453


In [27]:

# define the model
model = GradientBoostingClassifier(n_estimators=50)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 8.591532206535339
score_time 0.03588522672653198
test_acc 0.9243735530717248
test_recall 0.3057525252525253
test_auc 0.8939827227240169
test_precison 0.6861248035527516
test_f1 0.42063971420234514


In [80]:
# define the model with default hyperparameters
model = GradientBoostingClassifier()
# define the grid of values to search
grid = dict()

grid['random_state'] = [1]
grid['n_estimators'] = [100] #100, 300
grid['learning_rate'] = [0.5]#0.1, 1
# grid['max_samples'] = [0.7, 1] #0.5, 0.75
grid['max_features'] = [ None] #10, sqrt, 20, 40
grid['max_depth'] = [13] #3,6, 9, 11, 17, 21
#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit='f1')
# execute the grid search
grid_result = grid_search.fit(X_imp_st, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.476421 using {'learning_rate': 0.5, 'max_depth': 13, 'max_features': None, 'n_estimators': 300, 'random_state': 1}


In [82]:
#create df for metrics
columns= ['params', 'mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']
indexes = [x for x in range(len(grid_result.cv_results_['params']))]
metric_df = pd.DataFrame(index = indexes, columns=columns)
metric_df = metric_df.fillna(0)
metric_df['params'] = metric_df['params'].astype(str)
metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']] = metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']].astype(float)

metric_df['params'] = grid_result.cv_results_['params']
metric_df['mean_test_acc'] = grid_result.cv_results_['mean_test_acc']
metric_df['mean_test_recall'] = grid_result.cv_results_['mean_test_recall']
metric_df['mean_test_auc'] = grid_result.cv_results_['mean_test_auc']
metric_df['mean_test_precison'] = grid_result.cv_results_['mean_test_precison']
metric_df['mean_test_f1'] = grid_result.cv_results_['mean_test_f1']

metric_df.sort_values(by='mean_test_f1', ascending=False)

,params,mean_test_acc,mean_test_auc,mean_test_recall,mean_test_precison,mean_test_f1
0,"{'learning_rate': 0.5, 'max_depth': 13, 'max_features': None, 'n_estimators': 300, 'random_state': 1}",0.927275,0.893085,0.369581,0.678515,0.476421


In [28]:

# define the model
model = AdaBoostClassifier(n_estimators=50)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 3.20976243019104
score_time 0.1261271834373474
test_acc 0.9163935431005686
test_recall 0.3137878787878788
test_auc 0.8803571543909863
test_precison 0.5659285468775848
test_f1 0.402212959040932


In [116]:
# define the model with default hyperparameters
model = AdaBoostClassifier()
# define the grid of values to search
grid = dict()

grid['random_state'] = [1]
grid['n_estimators'] = [100] #100, 300
grid['learning_rate'] = [0.7] #0.01, 0.1, 0.2 , 0.3
# grid['max_features'] = [10, 20]
# grid['max_depth'] = [6] #3,9
#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit='f1')
# execute the grid search
grid_result = grid_search.fit(X_imp_st, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.412003 using {'learning_rate': 0.7, 'n_estimators': 100, 'random_state': 1}


In [117]:
#create df for metrics
columns= ['params', 'mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']
indexes = [x for x in range(len(grid_result.cv_results_['params']))]
metric_df = pd.DataFrame(index = indexes, columns=columns)
metric_df = metric_df.fillna(0)
metric_df['params'] = metric_df['params'].astype(str)
metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']] = metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']].astype(float)

metric_df['params'] = grid_result.cv_results_['params']
metric_df['mean_test_acc'] = grid_result.cv_results_['mean_test_acc']
metric_df['mean_test_recall'] = grid_result.cv_results_['mean_test_recall']
metric_df['mean_test_auc'] = grid_result.cv_results_['mean_test_auc']
metric_df['mean_test_precison'] = grid_result.cv_results_['mean_test_precison']
metric_df['mean_test_f1'] = grid_result.cv_results_['mean_test_f1']

metric_df.sort_values(by='mean_test_f1', ascending=False)

,params,mean_test_acc,mean_test_auc,mean_test_recall,mean_test_precison,mean_test_f1
0,"{'learning_rate': 0.7, 'n_estimators': 100, 'r...",0.918207,0.88344,0.318364,0.58752,0.412003


In [29]:

# define the model
model = BaggingClassifier(n_estimators=50)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 28.772070348262787
score_time 0.10915983915328979
test_acc 0.9281361424789347
test_recall 0.3388686868686869
test_auc 0.8865379112872658
test_precison 0.7144312129010065
test_f1 0.45806358937354374


In [61]:
# define the model with default hyperparameters
model = BaggingClassifier()
# define the grid of values to search
grid = dict()

grid['random_state'] = [1]
grid['n_estimators'] = [100] #100, 300
# grid['learning_rate'] = [0.3] #0.01, 0.1, 0.2 , 0.3
# grid['max_samples'] = [0.7, 1] #0.5, 0.75
# grid['max_features'] = [10, 20]
# grid['max_depth'] = [6] #3,9
#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit='f1')
# execute the grid search
grid_result = grid_search.fit(X_imp_st, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.458433 using {'n_estimators': 100, 'random_state': 1}


In [1]:
#create df for metrics
columns= ['params', 'mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']
indexes = [x for x in range(len(grid_result.cv_results_['params']))]
metric_df = pd.DataFrame(index = indexes, columns=columns)
metric_df = metric_df.fillna(0)
metric_df['params'] = metric_df['params'].astype(str)
metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']] = metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']].astype(float)

metric_df['params'] = grid_result.cv_results_['params']
metric_df['mean_test_acc'] = grid_result.cv_results_['mean_test_acc']
metric_df['mean_test_recall'] = grid_result.cv_results_['mean_test_recall']
metric_df['mean_test_auc'] = grid_result.cv_results_['mean_test_auc']
metric_df['mean_test_precison'] = grid_result.cv_results_['mean_test_precison']
metric_df['mean_test_f1'] = grid_result.cv_results_['mean_test_f1']

metric_df.sort_values(by='mean_test_f1', ascending=False)

NameError: name 'grid_result' is not defined

In [31]:

# define the model
model = BernoulliNB()

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 0.039883089065551755
score_time 0.02739964723587036
test_acc 0.863664432754754
test_recall 0.5547222222222222
test_auc 0.8423321752216889
test_precison 0.34255212411723546
test_f1 0.4231345204252966


In [49]:
# define the model with default hyperparameters
model = BernoulliNB()
# define the grid of values to search
grid = dict()
grid['alpha'] = [0.5] #0, 0.5, 1, 10
grid['binarize'] = [1,] #0, 0.1, 0.5,  1.5, 2, 5, 10
# grid['n_estimators'] = [500] #100, 300
# grid['learning_rate'] = [0.3] #0.01, 0.1, 0.2 , 0.3
# grid['subsample'] = [1] #0.5, 0.75
# # grid['max_features'] = [10, 20, 30]
# grid['max_depth'] = [6] #3,9
#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit='f1')
# execute the grid search
grid_result = grid_search.fit(X_imp_st, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Fram

Best: 0.457720 using {'alpha': 0.5, 'binarize': 1}


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
#create df for metrics
columns= ['params', 'mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']
indexes = [x for x in range(len(grid_result.cv_results_['params']))]
metric_df = pd.DataFrame(index = indexes, columns=columns)
metric_df = metric_df.fillna(0)
metric_df['params'] = metric_df['params'].astype(str)
metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']] = metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']].astype(float)

metric_df['params'] = grid_result.cv_results_['params']
metric_df['mean_test_acc'] = grid_result.cv_results_['mean_test_acc']
metric_df['mean_test_recall'] = grid_result.cv_results_['mean_test_recall']
metric_df['mean_test_auc'] = grid_result.cv_results_['mean_test_auc']
metric_df['mean_test_precison'] = grid_result.cv_results_['mean_test_precison']
metric_df['mean_test_f1'] = grid_result.cv_results_['mean_test_f1']

metric_df.sort_values(by='mean_test_f1', ascending=False)

,params,mean_test_acc,mean_test_auc,mean_test_recall,mean_test_precison,mean_test_f1
0,"{'alpha': 0.5, 'binarize': 1}",0.888329,0.835209,0.522091,0.408322,0.457720
1,"{'alpha': 0.5, 'binarize': 1.5}",0.890505,0.826087,0.493409,0.412844,0.448872
2,"{'alpha': 0.5, 'binarize': 2}",0.890188,0.801792,0.470247,0.407203,0.435752
3,"{'alpha': 0.5, 'binarize': 5}",0.912041,0.597661,0.131737,0.549244,0.212092
4,"{'alpha': 0.5, 'binarize': 10}",0.909775,0.517094,0.003025,0.275000,0.005980


In [79]:
# define the base models
level0 = list()
level0.append(('bernoulli', BernoulliNB(alpha=0.5, binarize=1)))
# level0.append(('knn',  KNeighborsClassifier(n_neighbors=5, weights='distance')))
#level0.append(('bagging', BaggingClassifier(n_estimators=100)))
# level0.append(('dectree', DecisionTreeClassifier()))
level0.append(('svc', SVC(C=10, gamma='scale', kernel='rbf')))
# level0.append(('gaussian', GaussianNB()))
# level0.append(('ada', AdaBoostClassifier(learning_rate=0.7, n_estimators=100,)))
level0.append(('gradient', GradientBoostingClassifier(learning_rate=0.5, max_depth=13, n_estimators=100, random_state=1)))
level0.append(('xgb', XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)))

# define meta learner model
level1 = XGBClassifier()
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

In [ ]:
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

In [158]:
from sklearn.ensemble import VotingClassifier

#scores = [0.457720, 0.458433, 0.412003, 0.476421, 0.50272, 0.4567053]
scores = [0.888329, 0.928136, 0.918207, 0.927275, 0.929905, 0.88130]

bernoulli = BernoulliNB(alpha=0.5, binarize=1)
bagging = BaggingClassifier(n_estimators=100)
adab = AdaBoostClassifier(learning_rate=0.7, n_estimators=100)
grad = GradientBoostingClassifier(learning_rate=0.5, max_depth=13, n_estimators=100, random_state=1)
XGB = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)
gaussian =GaussianNB()

estimators_list = [("bernoulli", bernoulli), ("bagging", bagging),\
    ("adab", adab), ("grad", grad), ("XGB", XGB), ("gaussian", gaussian)]

# create the ensemble
model = VotingClassifier(estimators=estimators_list, voting='soft', weights=scores)


In [159]:
# # # define the model
# model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=50, random_state=1,\
#     subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 358.7804761528969
score_time 1.3532228469848633
test_acc 0.9284535411589905
test_recall 0.4470656565656565
test_auc 0.8894185446397337
test_precison 0.652494544419745
test_f1 0.5294258929344249


In [160]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 358.7804761528969
score_time 1.3532228469848633
test_acc 0.9284535411589905
test_recall 0.4470656565656565
test_auc 0.8894185446397337
test_precison 0.652494544419745
test_f1 0.5294258929344249


Syntetic data generalization

In [189]:
X_train = X_imp_st.copy()
X_train['target'] = y

positives = X_train[X_train.target == 1]

In [193]:
from sdv.tabular import GaussianCopula
model = GaussianCopula()
model.fit(positives)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:621: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:5003: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2494: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/optim

In [194]:
sample = model.sample(2000)

In [195]:
from sdv.evaluation import evaluate
evaluate(sample, positives, metrics=['CSTest', 'KSTest'], aggregate=False)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,CSTest,Chi-Squared,NaN,NaN,0.0,1.0,MAXIMIZE,"Cannot find fields of types ('boolean', 'categ..."
1,KSTest,Inverted Kolmogorov-Smirnov D statistic,0.860483,0.860483,0.0,1.0,MAXIMIZE,None


In [175]:
X_synt = X_train.append(sample)

In [176]:
X_synt['target'].value_counts()

1    10994
0    10034
Name: target, dtype: int64

In [177]:
y_synt = X_synt['target']
X_synt = X_synt.drop('target', axis = 1)

In [178]:
# define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_synt , y_synt, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 178.52159756422043
score_time 0.23091553449630736
test_acc 0.9605526154698127
test_recall 0.946971089420134
test_auc 0.9912300868257351
test_precison 0.9768890337118531
test_f1 0.9616851809204598


In [179]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 178.52159756422043
score_time 0.23091553449630736
test_acc 0.9605526154698127
test_recall 0.946971089420134
test_auc 0.9912300868257351
test_precison 0.9768890337118531
test_f1 0.9616851809204598


SMOTE

In [137]:
from imblearn.over_sampling import SVMSMOTE
#overfit the target label
over = SVMSMOTE(sampling_strategy=0.2)
X_res, y_res = over.fit_resample(X_imp_st, y)

In [52]:
# define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_res, y_res, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

In [53]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 77.5805233836174
score_time 0.08300092220306396
test_acc 0.9442272060085017
test_recall 0.8149800796812748
test_auc 0.9692155756393932
test_precison 0.8970385427932932
test_f1 0.8537391876316913


Predict

In [212]:
predict_df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/predict_df_temp3.csv")
predict_df = predict_df.drop('Unnamed: 0', axis = 1)

predict_df['Pace_main'] = round(predict_df['Acceleration'] * 0.45 + predict_df['Sprint_speed'] * 0.55, 0)
predict_df['Shooting_main'] = round(predict_df['Finishing'] * 0.45 + predict_df['Long_shots'] * 0.20 + predict_df['Penalties'] * 0.05 + \
     predict_df['Positioning'] * 0.05 + predict_df['Shot_power'] * 0.20 + predict_df['Volleys'] * 0.05, 0)
predict_df['Dribbling_main'] = round(predict_df['Agility'] * 0.1 + predict_df['Balance'] * 0.05 + predict_df['Ball_control'] * 0.35 + \
     predict_df['Dribbling'] * 0.5, 0)
predict_df['Defending_main'] = round(predict_df['Heading_accuracy'] * 0.1 + predict_df['Interceptions'] * 0.2 + predict_df['Marking'] * 0.3 + \
     predict_df['Sliding_tackle'] * 0.3 + predict_df['Standing_tackle'] * 0.3, 0)
predict_df['Passing_main'] = round(predict_df['Crossing'] * 0.2 + predict_df['Curve'] * 0.05 + predict_df['FK_accuracy'] * 0.05 + \
     predict_df['Long_passing'] * 0.15 + predict_df['Short_passing'] * 0.35 + predict_df['Vision'] * 0.25, 0)
predict_df['Physical_main'] = round(predict_df['Aggression'] * 0.2 + predict_df['Jumping'] * 0.05 + predict_df['Stamina'] * 0.25 + \
     predict_df['Strength'] * 0.5, 0)

gk_predict = predict_df[predict_df.Position == 'GK']
outfielder_predict = predict_df[predict_df.Position != 'GK']

outfielder_predict = outfielder_predict.drop(['GK_diving', 'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes',\
    'ClubInvolvedSaves', 'ClubInvolvedSaveRate', 'GKMatchPlayed', 'GKStarts', 'GKMin', 'GK90s', 'GKGoalAgainst', 'GKGoalAgainstPer90',\
        'GKSaves', 'GKSavePercent', 'GKWin', 'GKDraw', 'GKLost', 'GKCleanSheet', 'GKCleanSheetPercent'], axis = 1)

outfielder_predict = outfielder_predict.drop(['ShotsOnTargetPercent', 'ClubInvolvedShotsOnTargetRate', 'ClubInvolvedGoalPerShot',\
    'ShotsPer90', 'Shots', 'ClubInvolvedShotsPer90', 'ClubInvolvedShots'], axis = 1)

outfielder_predict = outfielder_predict.drop(['GoalsPerShot', 'ClubInovlvedPossession'], axis = 1)

outfielder_predict['ClubInvolvedPlayed'] = outfielder_predict['ClubInvolvedWin'] + outfielder_predict['ClubInvolvedDraw'] + outfielder_predict['ClubInvolvedLoose']
# team_df['ClubInvolvedPlayed'] = team_df['ClubInvolvedWin'] + team_df['ClubInvolvedDraw'] + team_df['ClubInvolvedLoose']
# outfielder_predict['ClubAge'] = team_df['ClubInvolvedAge']
outfielder_predict['ClubInvolvedWinPerMatch'] = outfielder_predict['ClubInvolvedWin'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedDrawPerMatch'] = outfielder_predict['ClubInvolvedDraw'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedLoosePerMatch'] = outfielder_predict['ClubInvolvedLoose'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalForPerMatch'] = outfielder_predict['ClubInvolvedGoalFor'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalAgainstPerMatch'] = outfielder_predict['ClubInvolvedGoalAgainst'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedPointsPerMatch'] = outfielder_predict['ClubInvolvedPoints'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubPointsPerMatch'] = team_df['ClubInvolvedPoints'] / team_df['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedAssistsPerMatch'] = outfielder_predict['ClubInvolvedAssists'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedNonPenaltyGoalPerMatch'] = outfielder_predict['ClubInvolvedNonPenaltyGoal'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedYellowCardPerMatch'] = outfielder_predict['ClubInvolvedYellowCard'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedRedCardPerMatch'] = outfielder_predict['ClubInvolvedRedCard'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedNumPlayersUsedPerMatch'] = outfielder_predict['ClubInvolvedNumPlayersUsed'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubNumPlayersUsedPerMatch'] = team_df['ClubInvolvedNumPlayersUsed'] / team_df['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedShotsOnTargetPerMatch'] = outfielder_predict['ClubInvolvedShotsOnTarget'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalPerShotOnTargetPerMatch'] = outfielder_predict['ClubInvolvedGoalPerShotOnTarget'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubGoalPerShotOnTargetPerMatch'] = team_df['ClubInvolvedGoalPerShotOnTarget'] / team_df['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedShotOnTargetAgainstUsedPerMatch'] = outfielder_predict['ClubInvolvedShotOnTargetAgainst'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubShotOnTargetAgainstUsedPerMatch'] = team_df['ClubInvolvedShotOnTargetAgainst'] / team_df['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedCleanSheetsPerMatch'] = outfielder_predict['ClubInvolvedCleanSheets'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['MatchesPlayedPerClubMatches'] = outfielder_predict['MatchesPlayed'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['StartsPerClubMatches'] = outfielder_predict['Starts'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['NonPenaltyGoalsPerMinutes'] = outfielder_predict['MatchesPlayed'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['GoalsPerMinutesPlayed'] = outfielder_predict['Goals'] / outfielder_predict['MinutesPlayed']
outfielder_predict['AssistsPerMinutesPlayed'] = outfielder_predict['Assists'] / outfielder_predict['MinutesPlayed']
outfielder_predict['NonPenaltyGoalsPerMinutesPlayed'] = outfielder_predict['NonPenaltyGoals'] / outfielder_predict['MinutesPlayed']
outfielder_predict['YellowCardsPerMinutesPlayed'] = outfielder_predict['YellowCards'] / outfielder_predict['MinutesPlayed']
outfielder_predict['RedCardsPerMinutesPlayed'] = outfielder_predict['RedCards'] / outfielder_predict['MinutesPlayed']
outfielder_predict['ShotsOnTargetPerMinutesPlayed'] = outfielder_predict['ShotsOnTarget'] / outfielder_predict['MinutesPlayed']

outfielder_predict = outfielder_predict.drop(['ClubInvolvedPlayed', 'ClubInvolvedWin', 'ClubInvolvedDraw',\
     'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints',\
               'ClubInvolvedAssists', 'ClubInvolvedNonPenaltyGoal', 'ClubInvolvedYellowCard', 'ClubInvolvedRedCard',\
                     'ClubInvolvedNumPlayersUsed',\
                         'ClubInvolvedShotsOnTarget', 'ClubInvolvedGoalPerShotOnTarget', 'ClubInvolvedShotOnTargetAgainst', 'ClubInvolvedCleanSheets',\
                              'MatchesPlayed', 'Starts', 'MatchesPlayed', 'Goals', 'Assists', 'NonPenaltyGoals', 'YellowCards',\
                                   'RedCards', 'ShotsOnTarget',  'GoalsPer90', 'AssistsPer90', 'GoalsAssistsPer90', 'ShotsOnTargetPer90'], axis=1)

outfielder_predict['CorrMarketValue'] = outfielder_predict['NewMarketValue']

result_df = outfielder_predict[['Name', 'ClubInvolved', 'MainPosition',  'ID', 'Nationality']]

outfielder_predict = outfielder_predict.drop([ 'Name', 'ContractGood', 'Player',\
    'Nation', 'FbrefClub', 'GoalPerShotOnTarget', 'Pos', 'MarketValue', 'Composure', 'ID', 'Joined', 'NewMarketValue', 'Nationality'], axis=1)

outfielder_predict = outfielder_predict[feature_list]

In [213]:
#encode
outfielder_predict_cbe = cbe_encoder.transform(outfielder_predict)

In [214]:
#impute
outfielder_predict_cbe_imp = imputer.transform(outfielder_predict_cbe)
outfielder_predict_cbe_imp = pd.DataFrame(data = outfielder_predict_cbe_imp, columns = outfielder_predict_cbe.columns)

In [215]:
#standardization
outfielder_predict_imp_st = scaler.transform(outfielder_predict_cbe_imp)
outfielder_predict_imp_st = pd.DataFrame(data = outfielder_predict_imp_st, columns = outfielder_predict_cbe_imp.columns)

In [216]:
outfielder_predict_imp_st['LeagueInvolved_CorrMarketValue'] = outfielder_predict_imp_st['LeagueInvolved'] * outfielder_predict_imp_st['CorrMarketValue']
outfielder_predict_imp_st['LeagueInvolved_MinutesPlayedPer90'] = outfielder_predict_imp_st['LeagueInvolved'] * outfielder_predict_imp_st['MinutesPlayedPer90']
outfielder_predict_imp_st['LeagueInvolved_Season'] = outfielder_predict_imp_st['LeagueInvolved'] / outfielder_predict_imp_st['Season']
outfielder_predict_imp_st['Season_SP_LA_LIGA'] = outfielder_predict_imp_st['Season'] / outfielder_predict_imp_st['SP_LA_LIGA']

In [217]:
#predict 
model.fit(X_imp_st, y)
result_df['confidence'] = model.predict_proba(outfielder_predict_imp_st)[:, 1]
result_df.rename(columns={'ClubInvolved': 'Team', 'MainPosition':'Position', 'confidence':'Probability'}, inplace=True)

#result_df.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/TransferPredict/transfer_predict_api/transferPredict/transferPredictApp/Predictions/Data/result_SP_res.csv")


#result_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/result_SP_voting.xlsx")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [60]:
result_df.head()

,Name,Team,Position,ID,Nationality,Probability
0,L. Messi,FC Barcelona,Forward,158023,Argentina,0.404927
1,Cristiano Ronaldo,Juventus,Forward,20801,Portugal,0.532061
3,K. De Bruyne,Manchester City,Midfielder,192985,Belgium,0.025140
4,Neymar Jr,Paris Saint-Germain,Forward,190871,Brazil,0.871285
5,R. Lewandowski,FC Bayern München,Forward,188545,Poland,0.562884


In [218]:
pos = len(result_df[(result_df.Name.isin(['R. De Paul',
'Matheus Cunha',
'Emerson',
'Eric García',
'S. Agüero',
'M. Depay',
'E. Camavinga',
'D. Alaba',
'Rafa Mir',
'T. Delaney',
'L. Augustinsson',
'E. Lamela',
'Diego Rico',
'A. Danjuma Groeneveld',
'J. Foyth',
'B. Dia',
'Manu Morlanes',
'A. Mandi',
'Y. Sabaly',
'Rochina',
'Monchu',
'Escudero',
'Javi Galán',
'F. Cervi',
'Rubén Sobrino',
'L. Dimata',
'Vadillo',
'Sergi Gómez',
'Miguelón',
'Aleix Vidal',
'A. Budimir',
'Ramalho',
'Cote',
'Kike García',
'Jesús Areso',
'Franquesa',
'Marcos André',
'J. Silva',
'S. Mitrović',
'M. Sylla',
'Omar Mascarell',
'Gumbau',
'Bigas',
'M. Hoppe',
'Kangin Lee',
'Jaume Costa',
'Fran García',
'R. Nteka',
'Falcao',
'I. Balliu'
'D. Wass', 'Ferran Torres', 'P. Aubameyang', 'Dani Raba', 'Lucas Pérez', 'E. Cömert'
'Jason']))&(result_df.Probability>=0.5)])

In [220]:
positives = len(result_df[(result_df.Probability>=0.5)])
recall = round(pos/50, 4)
precision = round(pos/positives, 4)

false_pos = positives - pos
false_neg = 50 - pos
acc = round((11702 - false_pos - false_neg) / 11702,4)

print(f'Recall: {recall}\nPrecision: {precision}\nAccuracy: {acc}')

Recall: 0.4
Precision: 0.0353
Accuracy: 0.9507
